In [1]:
import torchvision.datasets as datasets
import numpy as np
from sklearn.decomposition import PCA

C:\ProgramData\Miniconda3\envs\ml37\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [2]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)

In [3]:
filtered_mnist_train = []
for image in mnist_trainset:
    if image[1] == 0 or image[1] == 1:
        filtered_mnist_train.append(image)

In [4]:
X = [np.asarray(val[0]).flatten() for val in filtered_mnist_train]
Y = [val[1] for val in filtered_mnist_train]

In [5]:
pca = PCA(n_components=2)
X_PC = pca.fit_transform(X)

In [6]:
X_PC.shape

(12665, 2)